In [1]:
import os
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle as pkl

In [2]:
driving_video_root = '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/driving_videos'
source_image_root = '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/source_images/09202024'
driving_video_paths = glob.glob(os.path.join(driving_video_root, '*.mp4'))
source_image_paths = glob.glob(os.path.join(source_image_root, '*.jpg'))[:2]
print(len(driving_video_paths), len(source_image_paths))
print(source_image_paths)

12 2
['/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/source_images/09202024/Keli_lc.jpg', '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/source_images/09202024/Matt_lc.jpg']


In [5]:
driving_video_paths

['/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/driving_videos/subject25_cap.mp4',
 '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/driving_videos/subject25_tied_hair.mp4',
 '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/driving_videos/subject5_cap.mp4',
 '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/driving_videos/subject5_tied_hair.mp4',
 '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/driving_videos/ning_1.mp4',
 '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/driving_videos/ning_2.mp4',
 '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/driving_videos/ning_3.mp4',
 '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/driving_videos/ning_4.mp4',
 '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/driving_videos/ning_5.mp4',
 '/prj/qct/mmrd-cv/esper/Misc0002/dataset/liveportrait-augmentation/driving_videos/atalie.mp4',
 '

In [6]:
# Load a video into a numpy array
def load_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return np.array(frames)

# Load a source image into a numpy array
def load_source_image(image_path):
    image = cv2.imread(image_path)
    return image

# Resize the short side of the video to 512 pixels, and do center crop for the long side, the output size is 512x512
def center_crop_and_resize_video(video, rt=512):
    """
    Args:
        video: numpy array, shape (T, H, W, C)
        rt: int, the short side of the output video (resize to)
    Returns:
        cropped_video: numpy array, shape (T, 512, 512, C)
    """
    frame_num, height, width, _ = video.shape
    if height > width:
        new_height = rt
        new_width = int(width * rt / height)
    else:
        new_width = rt
        new_height = int(height * rt / width)
    resized_video = cv2.resize(video, (new_width, new_height))
    height, width, _ = resized_video.shape
    start_x = (width - rt) // 2
    start_y = (height - rt) // 2
    cropped_video = resized_video[start_y:start_y+rt, start_x:start_x+rt]
    return cropped_video

video = load_video(driving_video_paths[10])
source_image = load_source_image(source_image_paths[0])

: 

In [4]:
video.shape

(5425, 720, 720, 3)

In [ ]:
cropped_video = center_crop_and_resize_video(video)

In [ ]:
# Visualize the first frame of the cropped video
plt.imshow(cropped_video[0])
plt.show()

In [ ]:
# Concate the cropped video and the source image
def concate_video_and_source_image(video, source_image):
    """
    Args:
        video: numpy array, shape (T, L, L, C)
        source_image: numpy array, shape (L, 2L, C)
    Returns:
        concated_video: numpy array, shape (T, L, L, C)
    """
    video_height, video_width, _ = video.shape
    source_image_height, source_image_width, _ = source_image.shape
    if video_height > video_width:
        source_image_height = int(source_image_height * video_width / video_height)
        source_image_width = video_width
    else:
        source_image_width = int(source_image_width * video_height / video_width)
        source_image_height = video_height
    resized_source_image = cv2.resize(source_image, (source_image_width, source_image_height))
    concated_video = np.concatenate([video, resized_source_image], axis=0)
    return concated_video